#まとめ

* train (最初の 3 週間)
* valid A（第4週の前半）
* valid B（第4週の後半）
* all train (train + validA + validB)
* test (第5週の前半)
* LB (第5週の後半)


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#config

In [2]:
class CFG:
  VER = 'baseline_ver14'
  types = 'carts'

In [3]:
import numpy as np
import pandas as pd

import collections
from collections import Counter

import lightgbm as lgb
from sklearn.model_selection import GroupKFold
import pickle

import glob

import gc

# Data Load

In [4]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/otto-train-and-test-data-for-local-validation/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/'

df_train = pd.read_parquet(INPUT_DIR+'train.parquet')
df_val = pd.read_parquet(INPUT_DIR+'test.parquet')

print(len(df_train))
print(len(df_val))

163955180
7683577


In [5]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [6]:
#debug
#df_train = df_train.head(100000)
#df_val = df_val.head(100000)
#print(len(df_train))
#print(len(df_val))

# stage1. candidate 生成

# step1 sessionのごとにitem候補生成 : valid A 

In [7]:
%%time
VER = 6
DISK_PIECES = 4

def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
top_15_carts_orders = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_15_carts_orders_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_15_carts_orders.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_15_carts_orders_v{VER}_{k}.pqt') ) )

top_20_buy2buy = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_15_buy2buy_v{VER}_0.pqt') )


CPU times: user 54.1 s, sys: 3.47 s, total: 57.5 s
Wall time: 1min 3s


In [8]:
#人気
val_popular_aid = df_val.loc[df_val['type']==CFG.types,'aid'].value_counts().index.values[:20]

#def popular_candidate(df):
#  return val_popular_aid

In [9]:
# step1
import itertools

type_weight_multipliers = {0: 1, 1: 6, 2: 3}


def generate_candidate(df, k=50):

  # USER HISTORY AIDS AND TYPES
  aids=df.aid.tolist()
  types = df.type.tolist()
  # UNIQUE AIDS AND UNIQUE BUYS
  unique_aids = list(dict.fromkeys(aids[::-1] ))
  df = df.loc[(df['type']==1)|(df['type']==2)]
  unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
  # RERANK CANDIDATES USING WEIGHTS
  if len(unique_aids)>=20:
      weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
      aids_temp = Counter() 
      # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
      for aid,w,t in zip(aids,weights,types): 
          aids_temp[aid] += w * type_weight_multipliers[t]
      # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
      aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
      for aid in aids3: aids_temp[aid] += 0.1
      sorted_aids = [k for k,v in aids_temp.most_common(k)]
      return sorted_aids[:k]
  # USE "CART ORDER" CO-VISITATION MATRIX
  aids2 = list(itertools.chain(*[top_15_carts_orders[aid] for aid in unique_aids if aid in top_15_carts_orders]))
  # USE "BUY2BUY" CO-VISITATION MATRIX
  aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
  # RERANK CANDIDATES
  top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(k) if aid2 not in unique_aids] 
  result = unique_aids + top_aids2[:k]
  # USE TOP20 TEST ORDERS
  return (result + list(val_popular_aid)[:k])[:k]

In [10]:
from tqdm import tqdm
tqdm.pandas()

candidates = df_val.groupby('session').progress_apply(lambda x: generate_candidate(x)).explode()
# candidates
candidates

100%|██████████| 1801251/1801251 [23:28<00:00, 1278.70it/s]


session
11098528      11830
11098528    1732105
11098528     588923
11098528     884502
11098528    1157882
             ...   
12899778     110408
12899778     379804
12899778     623534
12899778    1068655
12899778    1761007
Length: 49078466, dtype: object

In [11]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,11830
1,11098528,1732105
2,11098528,588923
3,11098528,884502
4,11098528,1157882
...,...,...
49078461,12899778,110408
49078462,12899778,379804
49078463,12899778,623534
49078464,12899778,1068655


# step2 item特徴 : train + valid A

In [12]:
df_train

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
163955175,11098523,175715,1661119197,0
163955176,11098524,1088524,1661119198,0
163955177,11098525,182927,1661119199,0
163955178,11098526,510055,1661119199,0


In [13]:
%%time
item_features = pd.merge(df_train, df_val, on=['session','aid','type'], how='left').groupby('aid').agg({'aid': 'count', 'session': 'nunique', 'type': 'mean'})
item_features.columns = ['item_item_count', 'item_user_count', 'item_by_count']
#item_features.to_parquet('item_features.pqt')
item_features

CPU times: user 2min 40s, sys: 14.4 s, total: 2min 55s
Wall time: 2min 54s


,item_item_count,item_user_count,item_by_count
aid,,,
0,33,26,0.000000
1,32,29,0.031250
2,12,12,0.000000
3,1201,668,0.079933
4,140,97,0.035714
...,...,...,...
1855598,6,6,0.000000
1855599,9,8,0.000000
1855600,68,42,0.073529


このアイテムがトレイン内でクリック,cart,orderされた回数

In [14]:
%%time
item_click_count_features = df_train[df_train['type']==0].groupby('aid').agg({'session': 'count'})
item_click_count_features.columns = ['item_click_count_features']
#item_features.to_parquet('item_features.pqt')
item_click_count_features

item_cart_count_features = df_train[df_train['type']==1].groupby('aid').agg({'session': 'count'})
item_cart_count_features.columns = ['item_cart_count_features']
#item_features.to_parquet('item_features.pqt')
item_cart_count_features

item_order_count_features = df_train[df_train['type']==2].groupby('aid').agg({'session': 'count'})
item_order_count_features.columns = ['item_order_count_features']
#item_features.to_parquet('item_features.pqt')
item_order_count_features

CPU times: user 18.1 s, sys: 503 ms, total: 18.6 s
Wall time: 18.5 s


,item_order_count_features
aid,
3,15
10,1
11,2
14,3
16,1
...,...
1855589,1
1855592,3
1855594,54


In [15]:
item_features = item_features.merge(item_click_count_features, on=['aid'], how='outer').fillna(0)
item_features = item_features.merge(item_cart_count_features, on=['aid'], how='outer').fillna(0)
item_features = item_features.merge(item_order_count_features, on=['aid'], how='outer').fillna(0)
#item_features = item_features.drop_duplicates('aid')
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
item_features

,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features
aid,,,,,,
0,33,26,0.000000,33.0,0.0,0.0
1,32,29,0.031250,31.0,1.0,0.0
2,12,12,0.000000,12.0,0.0,0.0
3,1201,668,0.079933,1120.0,66.0,15.0
4,140,97,0.035714,135.0,5.0,0.0
...,...,...,...,...,...,...
1855598,6,6,0.000000,6.0,0.0,0.0
1855599,9,8,0.000000,9.0,0.0,0.0
1855600,68,42,0.073529,64.0,3.0,1.0


In [16]:
del item_click_count_features, item_cart_count_features, item_order_count_features
gc.collect

<function gc.collect(generation=2)>

# step3 user特徴 : valid A

In [17]:
user_features = df_val.groupby('session').agg({'session': 'count', 'aid': 'nunique', 'type': 'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
#user_features.to_parquet('user_features.pqt')
user_features

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000
...,...,...,...
12899774,1,1,0.000000
12899775,1,1,0.000000
12899776,1,1,0.000000


In [18]:
#del 
#gc.collect

# step4 user × item 特徴量生成 : valid A

このアイテムはユーザーによって既にクリック,cart,orderされていますか

In [19]:
#user(session) x item(aid)に対するaction(type)の有無(1/0)を特徴量にする
def create_action_flag(df, type_number=0, column_name="user_item_click_flag"):
    action_flag_df = df.loc[df["type"] == type_number, ["session", "aid"]].drop_duplicates()
    action_flag_df[column_name] = 1
    return action_flag_df

In [20]:
user_item_click_flag = create_action_flag(df_val, type_number=0, column_name="user_item_click_flag")
user_item_cart_flag = create_action_flag(df_val, type_number=1, column_name="user_item_cart_flag")
user_item_order_flag = create_action_flag(df_val, type_number=2, column_name="user_item_order_flag")

In [21]:
tmp = pd.merge(
    user_item_click_flag,
    user_item_cart_flag,
    on=["session", "aid"],
    how="outer"
)
user_by_item_features = pd.merge(
    tmp,
    user_item_order_flag,
    on=["session", "aid"],
    how="outer"
).fillna(0)
user_by_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag
0,11098528,11830,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0
3,11098530,409236,1.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0
...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0
5535986,12896465,1306971,0.0,0.0,1.0
5535987,12896768,1303029,0.0,0.0,1.0
5535988,12898765,73333,0.0,0.0,1.0


In [22]:
#df_val[df_val['type']==0].groupby('session').agg({'type': 'count'})

ユーザーはこの項目を複数回クリックしましたか? 幾つか

In [23]:
#click
user_item_click_count = df_val[df_val['type']==0].groupby(['session','aid']).agg({'aid': 'count'})
user_item_click_count.columns = ['user_item_click_count']
user_item_click_count = df_val[['session','aid']].merge(user_item_click_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_click_count

#cart
user_item_cart_count = df_val[df_val['type']==1].groupby(['session','aid']).agg({'aid': 'count'})
user_item_cart_count.columns = ['user_item_cart_count']
user_item_cart_count = df_val[['session','aid']].merge(user_item_cart_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_cart_count

#order
user_item_order_count = df_val[df_val['type']==2].groupby(['session','aid']).agg({'aid': 'count'})
user_item_order_count.columns = ['user_item_order_count']
user_item_order_count = df_val[['session','aid']].merge(user_item_order_count, how='left', on=['session','aid'])
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_item_order_count

user_item_count = user_item_click_count.merge(user_item_cart_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_item_order_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
user_item_count

,session,aid,user_item_click_count,user_item_cart_count,user_item_order_count
0,11098528,11830,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0
2,11098530,264500,2.0,0.0,0.0
10,11098530,409236,3.0,1.0,0.0
74,11098531,452188,2.0,0.0,1.0
...,...,...,...,...,...
112908356,12899774,33035,1.0,0.0,0.0
112908357,12899775,1743151,1.0,0.0,0.0
112908358,12899776,548599,1.0,0.0,0.0
112908359,12899777,384045,1.0,0.0,0.0


In [24]:
#oka × suzuki merge

user_item_features = user_by_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count
0,11098528,11830,1.0,0.0,0.0,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0,2.0,0.0,0.0
3,11098530,409236,1.0,1.0,0.0,3.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0,0.0,0.0,1.0
5535986,12896465,1306971,0.0,0.0,1.0,0.0,0.0,1.0
5535987,12896768,1303029,0.0,0.0,1.0,0.0,0.0,1.0
5535988,12898765,73333,0.0,0.0,1.0,0.0,0.0,1.0


ユーザーがすでにクリック,cart,orderしたアイテムの数

In [25]:
#click
user_per_item_click_count = df_val[df_val['type']==0].groupby('session').agg({'aid': 'count'})
user_per_item_click_count.columns = ['user_per_item_click_count']
user_per_item_click_count = df_val[['session','aid']].merge(user_per_item_click_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_click_count

#cart
user_per_item_cart_count = df_val[df_val['type']==1].groupby('session').agg({'aid': 'count'})
user_per_item_cart_count.columns = ['user_per_item_cart_count']
user_per_item_cart_count = df_val[['session','aid']].merge(user_per_item_cart_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_cart_count

#order
user_per_item_order_count = df_val[df_val['type']==2].groupby('session').agg({'aid': 'count'})
user_per_item_order_count.columns = ['user_per_item_order_count']
user_per_item_order_count = df_val[['session','aid']].merge(user_per_item_order_count, how='left', on='session')
#user_item_click_count.to_parquet('user_item_click_count.pqt')
user_per_item_order_count

user_item_count = user_per_item_click_count.merge(user_per_item_cart_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_per_item_order_count, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
user_item_count

,session,aid,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count
0,11098528,11830,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0
2,11098530,264500,5.0,1.0,0.0
10,11098530,409236,5.0,1.0,0.0
74,11098531,452188,20.0,0.0,4.0
...,...,...,...,...,...
112908356,12899774,33035,1.0,0.0,0.0
112908357,12899775,1743151,1.0,0.0,0.0
112908358,12899776,548599,1.0,0.0,0.0
112908359,12899777,384045,1.0,0.0,0.0


In [26]:
user_item_features = user_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features

,session,aid,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count
0,11098528,11830,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,11098529,1105029,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11098530,264500,1.0,0.0,0.0,2.0,0.0,0.0,5.0,1.0,0.0
3,11098530,409236,1.0,1.0,0.0,3.0,1.0,0.0,5.0,1.0,0.0
4,11098531,452188,1.0,0.0,1.0,2.0,0.0,1.0,20.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5535985,12896465,1551275,0.0,0.0,1.0,0.0,0.0,1.0,5.0,3.0,9.0
5535986,12896465,1306971,0.0,0.0,1.0,0.0,0.0,1.0,5.0,3.0,9.0
5535987,12896768,1303029,0.0,0.0,1.0,0.0,0.0,1.0,11.0,3.0,2.0
5535988,12898765,73333,0.0,0.0,1.0,0.0,0.0,1.0,5.0,5.0,4.0


最後にクリック,cart,orderされたアイテムにflag

In [27]:
'''
#click
user_item_click_last = df_val[df_val['type']==0].groupby('session').agg({'aid': 'last'})
user_item_click_last.columns = ['user_item_click_last']
user_item_click_last = df_val[['session','aid']].merge(user_item_click_last, how='left', on='session')
#user_item_click_last.to_parquet('user_item_click_last.pqt')
user_item_click_last

#cart
user_item_cart_last = df_val[df_val['type']==1].groupby('session').agg({'aid': 'last'})
user_item_cart_last.columns = ['user_item_cart_last']
user_item_cart_last = df_val[['session','aid']].merge(user_item_cart_last, how='left', on='session')
#user_item_cuser_item_cart_lastlick_count.to_parquet('user_item_cart_last.pqt')
user_item_cart_last

#order
user_item_order_last = df_val[df_val['type']==2].groupby('session').agg({'aid': 'last'})
user_item_order_last.columns = ['user_item_order_last']
user_item_order_last = df_val[['session','aid']].merge(user_item_order_last, how='left', on='session')
#user_item_order_order.to_parquet('user_item_order_order.pqt')
user_item_order_last

user_item_count = user_item_click_last.merge(user_item_cart_last, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.merge(user_item_order_last, on=['session','aid'], how='outer').fillna(0)
user_item_count = user_item_count.drop_duplicates(['session','aid'])
#user_item_count = user_item_count.rename(columns={'session':'user','aid':'item'})
user_item_count

user_item_features = user_item_features.merge(user_item_count, on=['session','aid'], how='outer').fillna(0)
user_item_features
'''

"\n#click\nuser_item_click_last = df_val[df_val['type']==0].groupby('session').agg({'aid': 'last'})\nuser_item_click_last.columns = ['user_item_click_last']\nuser_item_click_last = df_val[['session','aid']].merge(user_item_click_last, how='left', on='session')\n#user_item_click_last.to_parquet('user_item_click_last.pqt')\nuser_item_click_last\n\n#cart\nuser_item_cart_last = df_val[df_val['type']==1].groupby('session').agg({'aid': 'last'})\nuser_item_cart_last.columns = ['user_item_cart_last']\nuser_item_cart_last = df_val[['session','aid']].merge(user_item_cart_last, how='left', on='session')\n#user_item_cuser_item_cart_lastlick_count.to_parquet('user_item_cart_last.pqt')\nuser_item_cart_last\n\n#order\nuser_item_order_last = df_val[df_val['type']==2].groupby('session').agg({'aid': 'last'})\nuser_item_order_last.columns = ['user_item_order_last']\nuser_item_order_last = df_val[['session','aid']].merge(user_item_order_last, how='left', on='session')\n#user_item_order_order.to_parquet('u

In [28]:
#del user_by_item_features, user_per_item_cart_count, user_per_item_order_count, user_item_click_last, user_item_cart_last, user_item_order_last, user_item_count
#gc.collect

In [29]:
del user_by_item_features, user_per_item_cart_count, user_per_item_order_count, user_item_count
gc.collect

<function gc.collect(generation=2)>

# step5 : step1に step2,3,4を追加

In [30]:
candidates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49078466 entries, 0 to 49078465
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   session  int64 
 1   item     object
dtypes: int64(1), object(1)
memory usage: 748.9+ MB


In [31]:
candidates = reduce_mem_usage(candidates)
candidates.info()

Memory usage of dataframe is 748.88 MB
Memory usage after optimization is: 416.74 MB
Decreased by 44.4%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49078466 entries, 0 to 49078465
Data columns (total 2 columns):
 #   Column   Dtype   
---  ------   -----   
 0   session  int32   
 1   item     category
dtypes: category(1), int32(1)
memory usage: 416.7 MB


In [32]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825499 entries, 0 to 1855602
Data columns (total 6 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   item_item_count            int64  
 1   item_user_count            int64  
 2   item_by_count              float64
 3   item_click_count_features  float64
 4   item_cart_count_features   float64
 5   item_order_count_features  float64
dtypes: float64(4), int64(2)
memory usage: 97.5 MB


In [33]:
item_features = reduce_mem_usage(item_features)
item_features.info()

Memory usage of dataframe is 97.49 MB
Memory usage after optimization is: 45.26 MB
Decreased by 53.6%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825499 entries, 0 to 1855602
Data columns (total 6 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   item_item_count            int32  
 1   item_user_count            int32  
 2   item_by_count              float16
 3   item_click_count_features  float32
 4   item_cart_count_features   float16
 5   item_order_count_features  float16
dtypes: float16(3), float32(1), int32(2)
memory usage: 45.3 MB


In [34]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801251 entries, 11098528 to 12899778
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   user_user_count  int64  
 1   user_item_count  int64  
 2   user_buy_ratio   float64
dtypes: float64(1), int64(2)
memory usage: 55.0 MB


In [35]:
user_features = reduce_mem_usage(user_features)
user_features.info()

Memory usage of dataframe is 54.97 MB
Memory usage after optimization is: 24.05 MB
Decreased by 56.2%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801251 entries, 11098528 to 12899778
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   user_user_count  int16  
 1   user_item_count  int16  
 2   user_buy_ratio   float16
dtypes: float16(1), int16(2)
memory usage: 24.0 MB


In [36]:
user_item_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5535990 entries, 0 to 5535989
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   session                    int32  
 1   aid                        int32  
 2   user_item_click_flag       float64
 3   user_item_cart_flag        float64
 4   user_item_order_flag       float64
 5   user_item_click_count      float64
 6   user_item_cart_count       float64
 7   user_item_order_count      float64
 8   user_per_item_click_count  float64
 9   user_per_item_cart_count   float64
 10  user_per_item_order_count  float64
dtypes: float64(9), int32(2)
memory usage: 464.6 MB


In [37]:
user_item_features = reduce_mem_usage(user_item_features)
user_item_features.info()

Memory usage of dataframe is 464.60 MB
Memory usage after optimization is: 179.50 MB
Decreased by 61.4%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5535990 entries, 0 to 5535989
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   session                    int32  
 1   aid                        int32  
 2   user_item_click_flag       float16
 3   user_item_cart_flag        float16
 4   user_item_order_flag       float16
 5   user_item_click_count      float16
 6   user_item_cart_count       float16
 7   user_item_order_count      float16
 8   user_per_item_click_count  float16
 9   user_per_item_cart_count   float16
 10  user_per_item_order_count  float16
dtypes: float16(9), int32(2)
memory usage: 179.5 MB


In [38]:
%%time
candidates = candidates.rename(columns={'session':'user'})

#step2
#item_features = pd.read_parquet('item_features.pqt')
candidates = candidates.merge(item_features, left_on='item', right_index=True, how='left').fillna(-1)

#step3
#user_features = pd.read_parquet('user_features.pqt')
user_features = user_features.rename(columns={'session':'user'})
candidates = candidates.merge(user_features, left_on='user', right_index=True, how='left').fillna(-1)

#step4
#user_features = pd.read_parquet('user_features.pqt')
user_item_features = user_item_features.rename(columns={'session':'user'})
user_item_features = user_item_features.rename(columns={'aid':'item'})
candidates = candidates.merge(user_item_features, on=['user','item'], how='left').fillna(0)
candidates

CPU times: user 1min 16s, sys: 2.24 s, total: 1min 18s
Wall time: 1min 17s


,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,user_buy_ratio,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count
0,11098528,11830,32546.0,18362.0,0.175659,27910.0,3556.0,1081.0,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,11098528,1732105,10121.0,5565.0,0.280762,7798.0,1804.0,519.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,588923,23956.0,14293.0,0.124939,21609.0,1701.0,646.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,884502,30419.0,17385.0,0.135742,27203.0,2304.0,913.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,1157882,27559.0,15556.0,0.137329,24543.0,2248.0,767.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49078461,12899778,110408,378.0,268.0,0.079346,353.0,20.0,5.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078462,12899778,379804,693.0,352.0,0.073608,652.0,31.0,10.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078463,12899778,623534,376.0,231.0,0.148926,331.0,34.0,11.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078464,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
del item_features, user_features, user_item_features
gc.collect

<function gc.collect(generation=2)>

# step6 : 学習できる形にするgt:0,gt:1

In [40]:
tar = pd.read_parquet(INPUT_DIR+'test_labels.parquet')
tar = tar.loc[tar['type']==CFG.types]
tar = tar.explode('ground_truth')
tar = tar.rename(columns={'ground_truth':'item'})
tar['item'] = tar['item'].astype('int32')
tar[CFG.types] = 1
tar = tar.rename(columns={'session':'user'})
tar

,user,type,item,carts
1,11098528,carts,1199737,1
8,11098533,carts,108676,1
8,11098533,carts,1406660,1
8,11098533,carts,988295,1
8,11098533,carts,1118792,1
...,...,...,...,...
2212555,12899644,carts,822934,1
2212587,12899676,carts,35328,1
2212598,12899686,carts,1627951,1
2212626,12899713,carts,1097818,1


In [41]:
tar = tar[['user', 'item', CFG.types]]
tar

,user,item,carts
1,11098528,1199737,1
8,11098533,108676,1
8,11098533,1406660,1
8,11098533,988295,1
8,11098533,1118792,1
...,...,...,...
2212555,12899644,822934,1
2212587,12899676,35328,1
2212598,12899686,1627951,1
2212626,12899713,1097818,1


In [42]:
candidates = candidates.merge(tar,on=['user','item'],how='left').fillna(0)
candidates

,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,carts
0,11098528,11830,32546.0,18362.0,0.175659,27910.0,3556.0,1081.0,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,11098528,1732105,10121.0,5565.0,0.280762,7798.0,1804.0,519.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,588923,23956.0,14293.0,0.124939,21609.0,1701.0,646.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,884502,30419.0,17385.0,0.135742,27203.0,2304.0,913.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,1157882,27559.0,15556.0,0.137329,24543.0,2248.0,767.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49078461,12899778,110408,378.0,268.0,0.079346,353.0,20.0,5.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078462,12899778,379804,693.0,352.0,0.073608,652.0,31.0,10.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078463,12899778,623534,376.0,231.0,0.148926,331.0,34.0,11.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078464,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
candidates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49078466 entries, 0 to 49078465
Data columns (total 21 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user                       int32  
 1   item                       int64  
 2   item_item_count            float64
 3   item_user_count            float64
 4   item_by_count              float16
 5   item_click_count_features  float32
 6   item_cart_count_features   float16
 7   item_order_count_features  float16
 8   user_user_count            int16  
 9   user_item_count            int16  
 10  user_buy_ratio             float16
 11  user_item_click_flag       float16
 12  user_item_cart_flag        float16
 13  user_item_order_flag       float16
 14  user_item_click_count      float16
 15  user_item_cart_count       float16
 16  user_item_order_count      float16
 17  user_per_item_click_count  float16
 18  user_per_item_cart_count   float16
 19  user_per_item_order_count  float16
 20  

In [44]:
candidates = reduce_mem_usage(candidates)
candidates.info()

Memory usage of dataframe is 3650.78 MB
Memory usage after optimization is: 2714.68 MB
Decreased by 25.6%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49078466 entries, 0 to 49078465
Data columns (total 21 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user                       int32  
 1   item                       int32  
 2   item_item_count            float32
 3   item_user_count            float16
 4   item_by_count              float16
 5   item_click_count_features  float32
 6   item_cart_count_features   float16
 7   item_order_count_features  float16
 8   user_user_count            int16  
 9   user_item_count            int16  
 10  user_buy_ratio             float16
 11  user_item_click_flag       float16
 12  user_item_cart_flag        float16
 13  user_item_order_flag       float16
 14  user_item_click_count      float16
 15  user_item_cart_count       float16
 16  user_item_order_count      float16
 17  user_per_item_

In [45]:
candidates

,user,item,item_item_count,item_user_count,item_by_count,item_click_count_features,item_cart_count_features,item_order_count_features,user_user_count,user_item_count,...,user_item_click_flag,user_item_cart_flag,user_item_order_flag,user_item_click_count,user_item_cart_count,user_item_order_count,user_per_item_click_count,user_per_item_cart_count,user_per_item_order_count,carts
0,11098528,11830,32546.0,18368.0,0.175659,27910.0,3556.0,1081.0,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,11098528,1732105,10121.0,5564.0,0.280762,7798.0,1804.0,519.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11098528,588923,23956.0,14296.0,0.124939,21609.0,1701.0,646.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11098528,884502,30419.0,17392.0,0.135742,27203.0,2304.0,913.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11098528,1157882,27559.0,15552.0,0.137329,24543.0,2248.0,767.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49078461,12899778,110408,378.0,268.0,0.079346,353.0,20.0,5.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078462,12899778,379804,693.0,352.0,0.073608,652.0,31.0,10.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078463,12899778,623534,376.0,231.0,0.148926,331.0,34.0,11.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49078464,12899778,1068655,488.0,371.0,0.086060,452.0,30.0,6.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# stage2. ranker

# Training

In [46]:
#スライスずれるのでtargetは最後のカラムにしておく必要あり。
FEATURES = list(candidates.columns[2:-1])
FEATURES

['item_item_count',
 'item_user_count',
 'item_by_count',
 'item_click_count_features',
 'item_cart_count_features',
 'item_order_count_features',
 'user_user_count',
 'user_item_count',
 'user_buy_ratio',
 'user_item_click_flag',
 'user_item_cart_flag',
 'user_item_order_flag',
 'user_item_click_count',
 'user_item_cart_count',
 'user_item_order_count',
 'user_per_item_click_count',
 'user_per_item_cart_count',
 'user_per_item_order_count']

In [47]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
skf = GroupKFold(n_splits=4)
for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates[CFG.types], groups=candidates['user'] )):
    print(f'=====fold : {fold}=====')

    X_train = candidates.loc[train_idx, FEATURES]
    y_train = candidates.loc[train_idx, CFG.types]
    X_valid = candidates.loc[valid_idx, FEATURES]
    y_valid = candidates.loc[valid_idx, CFG.types]

    #参考　https://knuu.github.io/ltr_by_lightgbm.html

    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg', #
        'lambdarank_truncation_level': 10,
        'ndcg_eval_at': 20,
        'n_estimators': 10,
        'boosting_type': 'dart',
        'random_state': 0,
    }

    train = lgb.Dataset(X_train, y_train, group=candidates.loc[train_idx,:].groupby('user')['item'].agg('count').values ) #sessionごとに長さ異なること考慮(20ではない可能性あり)
    valid = lgb.Dataset(X_valid, y_valid, group=candidates.loc[valid_idx,:].groupby('user')['item'].agg('count').values ) #sessionごとに長さ異なること考慮(20ではない可能性あり)
     
     
    model = lgb.train(
        params, train, valid_sets=valid, 
        early_stopping_rounds=20, #50
        verbose_eval=10  # 10 round毎に metric を表示
    )
    
    pickle.dump(model, open(OUTPUT_DIR + f'lgb_model_{CFG.VER}_{fold}.pkl','wb'))

    del X_train,y_train,X_valid,y_valid, model
    gc.collect

=====fold : 0=====


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[10]	valid_0's ndcg@20: 0.962869
=====fold : 1=====
[10]	valid_0's ndcg@20: 0.963261
=====fold : 2=====
[10]	valid_0's ndcg@20: 0.963163
=====fold : 3=====
[10]	valid_0's ndcg@20: 0.962867


In [49]:
del candidates, df_train
gc.collect

<function gc.collect(generation=2)>